# Particle Swarm Optimisation for solving the Cartpole Problem

In this implementation, we use PSO to optimise the weights of the linear function approximator that's used to create the policy for the cartpole.v1 environment.

This method manages to solver the environment in 3.0 iterations (on average). This makes it highly competitive with the far more advanced implementations found on the leaderboard. However, you could make a case for saying that this algorithm requires 30 iterations (on average) to solve as we candidate solutions in the form of the swarm. Regardless, the lack of DNN and gradient calculations cause this implementation to find extremely quickly.

For more details of this implementation see here:

## Imports

In [77]:
import gym
from gym import envs
import numpy as np
import time

## Create optimiser

This is a standard version of PSO. It could be improved with parameters tunning, similar to that seen in learning rate optimisers. 

In [78]:
class ParticleSwarmOptimisation(object):
    
    def __init__(self, fitnessFunction, bounds, numParticles, omega, v, phiL, phiG):
        self.bestGlobalFitness = -np.inf              
        self.bestGlobalPos = []     
        self.fitnessFunction = fitnessFunction
        self.bounds = bounds 
        self.swarm = [] #Create swarm
        for i in range(numParticles):
            self.swarm.append(Particle(bounds, omega, v, phiL, phiG))

    def maxamise(self, maxIterations, target):
        bestGlobalFitness = -np.inf              
        bestGlobalPos = []                
        #Optimization loop
        for i in range(1, maxIterations):
            #Evaluate each particles fitness
            for p in self.swarm:
                fitness = p.evaluate(self.fitnessFunction)
                #Determine if current particle is new global best
                if fitness > bestGlobalFitness:
                    bestGlobalPos = p.bestPos.copy()
                    bestGlobalFitness = p.bestFitness
            #Update velocity and positions
            for p in self.swarm:
                p.updateVelocity(bestGlobalPos)
                p.updatePosition(self.bounds)     
            #Resample best to see if environment is solved
            bestGlobalFitness = self.fitnessFunction(bestGlobalPos, 100)
            print('Iteration: ' + str(i) + ' Global best: ' + str(bestGlobalFitness))
            if bestGlobalFitness > target:
                return i
        return i #failed to solve

class Particle:
    def __init__(self, bounds, omega, v, phiL, phiG):
        self.bestFitness = -np.inf     
        self.bestPos = []
        self.omega, self.phiL, self.phiG  = omega, phiL, phiG

        self.position =  np.random.uniform(low=bounds[0], high=bounds[1], size=8)
        self.velocity = np.random.uniform(low=-v, high=v, size=8)

    # evaluate current fitness
    def evaluate(self, fitnessFunction):
        fitness = fitnessFunction(self.position, 25)
        #update best position
        if fitness > self.bestFitness:
            self.bestPos = self.position.copy()
            self.bestFitness = fitness
        else:
            #Re-evaluate best 
            self.bestFitness = fitnessFunction(self.bestPos, 25)
        return self.bestFitness
                    
    # update new particle velocity
    def updateVelocity(self, globalBest):
        velLocal= self.phiL * np.random.rand((len(self.bestPos))) * (self.bestPos - self.position)
        velGlobal = self.phiG * np.random.rand((len(self.bestPos))) * (globalBest - self.position)
        self.velocity = self.omega * self.velocity + velLocal + velGlobal

    # update the particle position 
    def updatePosition(self,bounds):  
        self.position = self.position + self.velocity            
        self.position[self.position < bounds[0]] = bounds[0]
        self.position[self.position > bounds[1]] = bounds[1]        

## Create Fitness Function

The particles position is used as the weights in a linear layer which is past through a softmax fuction to create a stochastic policy.

Note the need for repeated sampling of the environment to get a good estimate of the policy.

In [79]:
class CartpoleFitness(object):

    def __init__(self, terminationStep):
        self.env = gym.make('CartPole-v1')
        self.terminationStep = terminationStep

    def policy(self, state, pos):
	    z = state.dot(pos)
	    exp = np.exp(z)
	    return exp/np.sum(exp)

    def evaluate(self, pos, evaluationIterations):
        policy = np.reshape(pos, (4,2))
        rewardTotal = 0
        for i in range(evaluationIterations):
            state = self.env.reset()
            step = 0
            while True:
                step += 1
                probs = self.policy(state, policy)
                action = np.random.choice(2,p=probs)
                state, reward, terminal, _ = self.env.step(action)
                rewardTotal += reward
                if terminal or step > self.terminationStep:
                    break

        return rewardTotal  / (i+1)

## Create experiment

Change optimiser parameters here

In [80]:
cartpole = CartpoleFitness(200)

weightSpaceBounds = 8
numberOfParticles = 10
momentum = 0.5
initialVelocityBounds = 0.2
localWeight = 2
globalWeight = 2

p = [weightSpaceBounds, numberOfParticles, momentum, initialVelocityBounds, localWeight, globalWeight]

timeTotal = 0
iterationTotal = 0
for i in range(1, 11):
    start = time.time()
    solver = ParticleSwarmOptimisation(cartpole.evaluate, (-p[0],p[0]), p[1], p[2], p[3], p[4], p[5])
    k = solver.maxamise(25, 195)
    iterationTotal += k
    end = time.time() - start
    print('Trail ' + str(i) + ' solved in ' + str(k) + ' iterations and ' + "{:.1f}".format(end) + ' seconds.')
    timeTotal += end
print('Solved in an average of ' + str(iterationTotal/i) + ' iterations, with an average of ' + "{:.1f}".format(timeTotal /i) + ' seconds per trail.')

Iteration: 1 Global best: 101.7
Iteration: 2 Global best: 164.81
Iteration: 3 Global best: 201.0
Trail 1 solved in 3 iterations and 6.5 seconds.
Iteration: 1 Global best: 87.65
Iteration: 2 Global best: 105.42
Iteration: 3 Global best: 192.22
Iteration: 4 Global best: 198.77
Trail 2 solved in 4 iterations and 7.9 seconds.
Iteration: 1 Global best: 136.89
Iteration: 2 Global best: 199.7
Trail 3 solved in 2 iterations and 3.4 seconds.
Iteration: 1 Global best: 190.28
Iteration: 2 Global best: 201.0
Trail 4 solved in 2 iterations and 5.1 seconds.
Iteration: 1 Global best: 135.67
Iteration: 2 Global best: 198.91
Trail 5 solved in 2 iterations and 3.5 seconds.
Iteration: 1 Global best: 175.69
Iteration: 2 Global best: 186.95
Iteration: 3 Global best: 194.69
Iteration: 4 Global best: 200.39
Trail 6 solved in 4 iterations and 12.7 seconds.
Iteration: 1 Global best: 86.83
Iteration: 2 Global best: 115.48
Iteration: 3 Global best: 146.22
Iteration: 4 Global best: 150.68
Iteration: 5 Global best

We can see that the cartpole problem is solved in an average of 3.0 iterations, with an average of 6.5 seconds per trail.

This average time could be greatly improved with the use of parallel computation and better hardware.